In [31]:
import requests
import pandas as pd
import time

#### 1) Pull all available daily and hourly data for the two big index ETFs, SPY and QQQ
####    A big nope on hourly - I thought technical data went back more than the first "slice" but no luck.
####    Going to expand on the number of ETFs for growing the dataset, but I could calculate the technical indicators for hourly data in the future (or find a source that likely isn't free)
#### 2) Generate the three rolling window sizes for these four datasets
#### 3) Calculate percent changes and standardization 12 new datasets – three for each ETF’s daily data and three for each ETF’s hourly data. It is expected that QQQ will have a higher standard deviation of percent changes, so this will be a good evaluation for the two standardization approaches.
#### 4) Pull, standardize, and merge all available daily and hourly technical indicator data for the two symbols with the respective OHLC datasets by date or time.
#### 5) Concatenate the four datasets for each rolling window.

In [73]:
# symbols and technical indicators [code, interval, name]
# https://www.alphavantage.co/documentation/#technical-indicators
symbol_list = ['SPY','QQQ','XLF','EEM','XLE','EWZ','IWM','SLV','FXI','HYG','GDX','EFA','TLT','LQD','XLU','XLV','XLI','IEMG','VWO','XLK','IEF','JNK','XLB','JETS','BND']
tech_list = [['SMA',50,'Technical Analysis: SMA'],
             ['EMA',21,'Technical Analysis: EMA'],
             ['RSI',14,'Technical Analysis: RSI']]

for symbol in symbol_list:

    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey=PDS8Y8E8KULJVDET"
    r = requests.get(url)
    data = r.json()
    df_price = pd.DataFrame(data['Time Series (Daily)']).T
    time.sleep(15)

    for tech in tech_list:
        url = f"https://www.alphavantage.co/query?function={tech[0]}&symbol={symbol}&interval=daily&time_period={tech[1]}&series_type=close&apikey=PDS8Y8E8KULJVDET"
        r = requests.get(url)
        data = r.json()
        df_tech = pd.DataFrame(data[tech[2]]).T
        df_price = df_price.merge(df_tech, how='inner', left_index=True, right_index=True)
        time.sleep(15)
    
    df_price.to_csv(f"../data/raw/{symbol}_daily.csv")
    print(f"{symbol} saved")


SPY saved
QQQ saved
XLF saved
EEM saved
XLE saved
EWZ saved
IWM saved
SLV saved
FXI saved
HYG saved
GDX saved
EFA saved
TLT saved
LQD saved
XLU saved
XLV saved
XLI saved
IEMG saved
VWO saved
XLK saved
IEF saved
JNK saved
XLB saved
JETS saved
BND saved
